Install dependencies

In [ ]:
!pip install pandas
!pip install spotipy

Import requirements

In [ ]:
import pandas as pd
import spotipy as sp
import glob

from spotipy.oauth2 import SpotifyClientCredentials

Initialize Spotify client

In [3]:
sp_id="<Insert your id here>"
sp_secret="<Insert your secret here>"

sp_client = sp.Spotify(auth_manager=SpotifyClientCredentials(client_id=sp_id, client_secret=sp_secret))

Function to download track features from the Spotify API

In [ ]:
def get_audio_features(track_id: str):
    result = sp_client.audio_features(track_id)
    if result == [None]:
        print("{} can't be retrieved".format(track_id))
        return pd.DataFrame() # return empty frame
    return pd.DataFrame.from_records(result)

Main loop

In [ ]:

# Read all files in path
path = r'data'
all_files = glob.glob(path + "/chart_*.csv")

track_ids = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    if df.shape[0] > 0:
        track_ids += df["track_id"].unique().tolist()

unique_track_ids = list(set(track_ids))

print("Downloading {} track feature sets. This may take some time.".format(len(unique_track_ids)))

track_frame = pd.DataFrame()
for track_id in unique_track_ids:
    track_frame = track_frame.append(get_audio_features(track_id))

# Move track id to the front and rename
track_frame_ids = track_frame["id"]
track_frame = track_frame.drop("id", 1)
track_frame.insert(0, 'track_id', track_frame_ids)

# drop unneeded features
track_frame = track_frame.drop("type", 1)
track_frame = track_frame.drop("uri", 1)
track_frame = track_frame.drop("track_href", 1)
track_frame = track_frame.drop("analysis_url", 1)

# safe to file
track_frame.to_csv("data/track_features.csv", index=False)